In [ ]:
!pip install easyocr
!pip install openpyxl
!pip install pdf2image
!apt-get install poppler-utils
!pip install opencv-python

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (22.02.0-2ubuntu0.8).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [ ]:
from pdf2image import convert_from_path
import easyocr
import pandas as pd
import re

# Convert PDF to images (1st page assumed)
pdf_path = "/content/Sem_4.pdf"  # change if needed
pages = convert_from_path(pdf_path)
image_path = "/content/page1.png"
pages[0].save(image_path)

# Load curriculum and grade mapping
curriculum_df = pd.read_excel('/content/Curriculum.xlsx')  # Course Code, Course Name, Credits
grade_df = pd.read_excel('/content/Marks.xlsx')            # Grade, Marks (grade points)
grade_dict = dict(zip(grade_df['Grade'].str.strip(), grade_df['Marks']))

In [ ]:
import easyocr
import pandas as pd
import re

# Step 1: OCR from image
reader = easyocr.Reader(['en'])
ocr_result = reader.readtext('/content/page1.png', detail=0)

# Step 2: Clean OCR text
ocr_text = "\n".join(ocr_result)
print("📄 OCR Extracted Text:\n", ocr_text)

lines = [line.strip() for line in ocr_text.split('\n') if line.strip()]

# Step 3: Extract after "Grade"
try:
    start_index = lines.index("Grade") + 1
except ValueError:
    raise Exception("Header 'Grade' not found in OCR output.")

lines = lines[start_index:]

# Step 4: Group into 4-line records
records = [lines[i:i+4] for i in range(0, len(lines), 4)]

# Step 5: Parse each record
structured_data = []
grades = []
for rec in records:
    if len(rec) == 4:
        sl_no, code, name, grade = rec

        # Fix course code: Replace letter O with digit 0
        code = code.strip().replace('O', '0')

        # Fix grade: Replace digit 0 with letter O
        grade = grade.strip().replace('0', 'O')

        # Fix misread course codes like CS200 → CS2000
        if re.match(r'CS\d{3}$', code):
            code += '0'

        structured_data.append((code, name.strip(), grade))
        grades.append(grade)

# Step 6: Show results
print("\n✅ Extracted Grades:", grades)
df_structured = pd.DataFrame(structured_data, columns=["Course Code", "Course Name", "Grade"])
print("\n📊 Structured Data:")
print(df_structured)


📄 OCR Extracted Text:
 Sl No:
Course Code
Course Name
Grade
1
CS1501
Foundations of Data Science
A+
2
CS200
Agile Scrum Process
0
3
CS2002
Computer Networks
A+
4
CS2004
Design and Analysis of Algorithms
A
5
CS2006
Introduction to Sensor Technology
A
6
CS2008
Operating Systems
A
7
CS2800
Design Thinking Lab
B+
8
CS2802
Computer Networks Lab
A+
9
CS2804
Operating Systems Lab
A+
10
EAIOOIT
Extra Academic Activity

✅ Extracted Grades: ['A+', 'O', 'A+', 'A', 'A', 'A', 'B+', 'A+', 'A+']

📊 Structured Data:
  Course Code                        Course Name Grade
0      CS1501        Foundations of Data Science    A+
1      CS2000                Agile Scrum Process     O
2      CS2002                  Computer Networks    A+
3      CS2004  Design and Analysis of Algorithms     A
4      CS2006  Introduction to Sensor Technology     A
5      CS2008                  Operating Systems     A
6      CS2800                Design Thinking Lab    B+
7      CS2802              Computer Networks Lab    A+

In [ ]:
total_points = 0
total_credits = 0

print("📊 Course-wise Credit and Grade Points:\n")

for code, name, grade in structured_data:
    credit_row = curriculum_df[curriculum_df['Course Code'].str.strip() == code.strip()]
    if not credit_row.empty and grade in grade_dict:
        credits = credit_row['Credits'].values[0]
        grade_point = grade_dict[grade]
        total_points += credits * grade_point
        total_credits += credits
        print(f"{code}: {name} | Grade: {grade} | Credits: {credits} | Points: {credits * grade_point}")
    else:
        print(f"⚠️ Skipping {code} - Missing credit or grade mapping.")

# Final GPA
gpa = total_points / total_credits if total_credits > 0 else 0
print(f"\n🎓 Final GPA: {round(gpa, 2)}")


📊 Course-wise Credit and Grade Points:

CS1501: Foundations of Data Science | Grade: A+ | Credits: 3 | Points: 27
CS2000: Agile Scrum Process | Grade: O | Credits: 1 | Points: 10
CS2002: Computer Networks | Grade: A+ | Credits: 3 | Points: 27
CS2004: Design and Analysis of Algorithms | Grade: A | Credits: 3 | Points: 24
CS2006: Introduction to Sensor Technology | Grade: A | Credits: 3 | Points: 24
CS2008: Operating Systems | Grade: A | Credits: 3 | Points: 24
CS2800: Design Thinking Lab | Grade: B+ | Credits: 1 | Points: 7
CS2802: Computer Networks Lab | Grade: A+ | Credits: 2 | Points: 18
CS2804: Operating Systems Lab | Grade: A+ | Credits: 2 | Points: 18

🎓 Final GPA: 8.52


In [ ]:
# Convert PDF to images (1st page assumed)
pdf_path = "/content/Sem_3.pdf"  # change if needed
pages = convert_from_path(pdf_path)
image_path = "/content/page2.png"
pages[0].save(image_path)

In [ ]:
import easyocr
import pandas as pd
import re

# Step 1: OCR from image
reader = easyocr.Reader(['en'])
ocr_result = reader.readtext('/content/page2.png', detail=0)

# Step 2: Clean OCR text
ocr_text = "\n".join(ocr_result)
print("📄 OCR Extracted Text:\n", ocr_text)

lines = [line.strip() for line in ocr_text.split('\n') if line.strip()]

# Step 3: Extract after "Grade"
try:
    start_index = lines.index("Grade") + 1
except ValueError:
    raise Exception("Header 'Grade' not found in OCR output.")

lines = lines[start_index:]

# Step 4: Group into 4-line records
records = [lines[i:i+4] for i in range(0, len(lines), 4)]

# Step 5: Parse each record
structured_data = []
grades = []
for rec in records:
    if len(rec) == 4:
        sl_no, code, name, grade = rec

        # Fix course code: Replace letter O with digit 0
        code = code.strip().replace('O', '0')

        # Fix grade: Replace digit 0 with letter O
        grade = grade.strip().replace('0', 'O')

        structured_data.append((code, name.strip(), grade))
        grades.append(grade)

# Step 6: Show results
print("\n✅ Extracted Grades:", grades)
df_structured = pd.DataFrame(structured_data, columns=["Course Code", "Course Name", "Grade"])
print("\n📊 Structured Data:")
print(df_structured)


📄 OCR Extracted Text:
 Sl No:
Course Code
Course Name
Grade
1
CS1OO6T
Data Structures
A+
2
CS1804T
Data Structures Lab
A
3
CS2OO1T
Database Management Systems
A
4
CS2003
Object Oriented Programming
A
5
CS2005
Software Engineering and Design
A+
6
CS28O1T
Database Management Systems Lab
0
7
MA2OO1
Discrete Mathematics and Graph
A+
Theory

✅ Extracted Grades: ['A+', 'A', 'A', 'A', 'A+', 'O', 'A+']

📊 Structured Data:
  Course Code                      Course Name Grade
0     CS1006T                  Data Structures    A+
1     CS1804T              Data Structures Lab     A
2     CS2001T      Database Management Systems     A
3      CS2003      Object Oriented Programming     A
4      CS2005  Software Engineering and Design    A+
5     CS2801T  Database Management Systems Lab     O
6      MA2001   Discrete Mathematics and Graph    A+


In [ ]:
total_points = 0
total_credits = 0

print("📊 Course-wise Credit and Grade Points:\n")

for code, name, grade in structured_data:
    credit_row = curriculum_df[curriculum_df['Course Code'].str.strip() == code.strip()]
    if not credit_row.empty and grade in grade_dict:
        credits = credit_row['Credits'].values[0]
        grade_point = grade_dict[grade]
        total_points += credits * grade_point
        total_credits += credits
        print(f"{code}: {name} | Grade: {grade} | Credits: {credits} | Points: {credits * grade_point}")
    else:
        print(f"⚠️ Skipping {code} - Missing credit or grade mapping.")

# Final GPA
gpa = total_points / total_credits if total_credits > 0 else 0
print(f"\n🎓 Final GPA: {round(gpa, 2)}")

📊 Course-wise Credit and Grade Points:

CS1002: Programming in Python | Grade: A+ | Credits: 2 | Points: 18
⚠️ Skipping CS1004 - Missing credit or grade mapping.
CS1702: Introduction to Internet of Things + Lab | Grade: A+ | Credits: 4 | Points: 36
CS1802: Programming in Python Lab | Grade: A | Credits: 2 | Points: 16
EN1002: English for Engineers | Grade: A+ | Credits: 3 | Points: 27
MA1002: Probability and Statistics | Grade: A | Credits: 3 | Points: 24
PH1001T: Engineering Physics` | Grade: A | Credits: 3 | Points: 24
PH1801T: Engineering Physics Lab | Grade: A+ | Credits: 2 | Points: 18

🎓 Final GPA: 8.58
